In [11]:
# запускаем в colab или локально?
try:
    from google.colab import drive
    colab = True
except ImportError:
    colab = False

print(f"colab: {colab}")

colab: False


In [15]:
# установка необходимых пакетов в colab
if colab:
    ! pip install rootutils -q
    ! pip install torchmetrics -q
    ! pip install livelossplot -q
    ! pip install torchinfo -q

In [13]:
# монтирование google диска и установка
# рабочей директории в `computer-vision`

import os
import rootutils

root = rootutils.setup_root(".", indicator="homeworks", pythonpath=True)

if colab:
    drive.mount("/content/drive", force_remount=True)
    os.chdir("drive/MyDrive/computer-vision")
else:
    os.chdir(root)

print(f"working directory: {os.getcwd()}")

working directory: /Users/alex/computer-vision


In [14]:
# создание директории для данных

from pathlib import Path

if colab:
    DATA_DIR = Path("/content/data")
else:
    DATA_DIR = root / "data"

DATA_DIR.mkdir(exist_ok=True)

print(f"DATA_DIR: {DATA_DIR}")

DATA_DIR: /Users/alex/computer-vision/data


In [16]:
# настройки для matplotlib

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format="retina"

plt.style.use("seaborn-v0_8-notebook")

## **Полносвязные нейронные сети. Методы регуляризации**

### **Неглубокие нейронные сети**

Неглубокая нейронная сеть - это полносвязная сеть с одним скрытым слоем.

#### **Пример 1**

<center>
    <figure>
        <img src="figs/02/shallow_net1.png" width=70%"/>
        <img src="figs/02/relu.png" width=20%"/>
    </figure>
</center>

- Вычисление скрытого слоя:

    $$
        \begin{pmatrix}
        h_1 \\ h_2 \\ h_3
        \end{pmatrix} =
        \mathbf{a}\left[
        \begin{pmatrix}
                \theta_{10} \\ \theta_{20} \\ \theta_{30}
        \end{pmatrix} +
        \begin{pmatrix}
                \theta_{11} \\ \theta_{21} \\ \theta_{31}
        \end{pmatrix}
        \cdot x
        \right]
        \qquad\rightarrow\qquad
        \mathbf{h} = \mathbf{a}\left[\boldsymbol{\beta}_0 + \mathbf{\Omega}_0 x\right]
    $$
    где $\mathbf{a}$ - функция активации, обычно это $\mathrm{ReLU}$ (Rectified Linear Unit):
    $$
        a[z] = \mathrm{ReLU}[z] =
        \begin{cases}
            0 & z < 0\\
            z & z \geqslant 0
        \end{cases}
    $$

- Вычисления выходного слоя:
    $$
        y = \phi_0 +
        \begin{pmatrix}
                \phi_1\,, & \phi_2\,, & \phi_3
        \end{pmatrix}
        \cdot
        \begin{pmatrix}
            h_1 \\ h_2 \\ h_3
        \end{pmatrix}
        \qquad\rightarrow\qquad
        y = \boldsymbol{\beta}_1 + \boldsymbol{\Omega}_1 \mathbf{h}
    $$

- Вычисления в сети в виде одной функции:
    $$
        y = f[x, \mathbf{w}] = \boldsymbol{\beta}_1 + \mathbf{\Omega}_1 \cdot
        \mathbf{a}\left[\boldsymbol{\beta}_0 + \mathbf{\Omega}_0 x\right]
    $$

Параметры нейронной сети 
$$
    \mathbf{w} = \left\{\boldsymbol{\beta}_0,\boldsymbol{\Omega}_0,\boldsymbol{\beta}_1,\boldsymbol{\Omega}_1\right\}
$$
У этой сети 10 параметров.

Реализация нейронной сети в PyTorch:

In [18]:
from torch import nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.lin1 = nn.Linear(
            in_features = 1,     # число нейронов во входном слое
            out_features = 3     # число нейронов в скрытом слое
        )

        self.lin2 = nn.Linear(
            in_features = 3,     # число нейронов в скрытом слое
            out_features = 1     # число нейронов в выходном слое
        )
        
        self.act = nn.ReLU()     # функция активации ReLU

    def forward(self, x):
        z = self.lin1(x)
        h = self.act(z)
        y = self.lin2(h)

        return y

Протестируем работу сети:

In [31]:
import torch

# модель сети
model = Net()

# батч размера 4 x 1 из случайных чисел
x = torch.rand((4, 1))

# forward pass
y = model(x)

print(f"x.shape: {x.shape}")
print(f"x: {x}")
print(f"y.shape: {y.shape}")
print(f"y: {y}")

x.shape: torch.Size([4, 1])
x: tensor([[0.2698],
        [0.4974],
        [0.6462],
        [0.3663]])
y.shape: torch.Size([4, 1])
y: tensor([[-0.4657],
        [-0.4657],
        [-0.4657],
        [-0.4657]], grad_fn=<AddmmBackward0>)


Выведем информацию о архитектуре сети:

In [33]:
import torchinfo

print(torchinfo.summary(model, input_size=x.shape, device="cpu"))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [4, 1]                    --
├─Linear: 1-1                            [4, 3]                    6
├─ReLU: 1-2                              [4, 3]                    --
├─Linear: 1-3                            [4, 1]                    4
Total params: 10
Trainable params: 10
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
